# Evaluate Fisher information

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os

import madminer.core
from madminer.fisherinformation import FisherInformation
from madminer.plotting import plot_fisher_information_contours_2d

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')

In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma/'
result_dir = base_dir + 'data/results/wgamma/'

## Settings

In [4]:
weight_settings = [None, 1., 10., 100.]
weight_setting_filenames = ['', '_w1', '_w10', '_w100']

## FisherInformation instances

In [6]:
fisher_all = FisherInformation(sample_dir + 'samples.h5', debug=True)
fisher_tight = FisherInformation(sample_dir + 'samples_tight.h5', debug=True)

14:56  
14:56  ------------------------------------------------------------
14:56  |                                                          |
14:56  |  MadMiner v2018.11.13                                    |
14:56  |                                                          |
14:56  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
14:56  |                                                          |
14:56  ------------------------------------------------------------
14:56  
14:56  Loading data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples.h5
14:56  Found 2 parameters:
14:56     cWWW (LHA: dim6 1, maximal power in squared ME: (2,), range: (-0.02, 0.02))
14:56     cWWWtilde (LHA: dim6 2, maximal power in squared ME: (2,), range: (-0.02, 0.02))
14:56  Found 6 benchmarks:
14:56     sm: cWWW = 0.00e+00, cWWWtilde = 0.00e+00
14:56     morphing_basis_vector_1: cWWW = -6.07e-03, cWWWtilde = -1.84e-02
14:56     morphing_basis_vector_2: 

## Truth-level info

In [6]:
fisher_info, cov = fisher_all.calculate_fisher_information_full_truth(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_parton.npy', fisher_info)
np.save(result_dir + 'information_covariance_parton.npy', cov)


In [7]:
fisher_info, cov = fisher_tight.calculate_fisher_information_full_truth(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_parton_tight.npy', fisher_info)
np.save(result_dir + 'information_covariance_parton_tight.npy', cov)


## Rate-only info

In [8]:
fisher_info, cov = fisher_all.calculate_fisher_information_rate(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_xsec.npy', fisher_info)
np.save(result_dir + 'information_covariance_xsec.npy', cov)


In [9]:
fisher_info, cov = fisher_tight.calculate_fisher_information_rate(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_xsec_tight.npy', fisher_info)
np.save(result_dir + 'information_covariance_xsec_tight.npy', cov)



## Full Fisher info (ML)

In [7]:
means, covs = fisher_all.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all',
    ensemble_vote_expectation_weight=weight_settings,
    test_split=0.5
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_full{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_full{}.npy'.format(weight_filename), cov)

TypeError: calculate_fisher_information_full_detector() got an unexpected keyword argument 'uncertainty'

In [11]:
means, covs = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all_tight',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_full_tight{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_full_tight{}.npy'.format(weight_filename), cov)
    

21:46  Evaluating rate Fisher information
21:47  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all_tight/ensemble.json
21:47  Found ensemble with 10 estimators and expectations [[ 0.0911617  -0.16387829]
 [ 0.13319454 -0.11376067]
 [ 0.02514527 -0.00822992]
 [-0.09863859  0.04448954]
 [ 0.22666614 -0.14302233]
 [ 0.13114008  0.00336235]
 [-0.08914433 -0.02287172]
 [-0.10113602 -0.02783563]
 [-0.01210352 -0.01167924]
 [ 0.11039541 -0.00848151]]
21:47  Evaluating kinematic Fisher information on batch 1 / 7
21:47  Evaluating kinematic Fisher information on batch 2 / 7
21:47  Evaluating kinematic Fisher information on batch 3 / 7
21:47  Evaluating kinematic Fisher information on batch 4 / 7
21:48  Evaluating kinematic Fisher information on batch 5 / 7
21:48  Evaluating kinematic Fisher information on batch 6 / 7
21:48  Evaluating kinematic Fisher information on batch 7 / 7


## Minimal observable set (ML)

In [12]:
means, covs = fisher_all.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_minimal',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_minimal{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_minimal{}.npy'.format(weight_filename), cov)

21:49  Evaluating rate Fisher information
21:49  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_minimal/ensemble.json
21:49  Found ensemble with 10 estimators and expectations [[ 0.00084352  0.02238362]
 [-0.01199366  0.01332431]
 [-0.00386162  0.00224334]
 [ 0.01450194  0.02153249]
 [ 0.02635764 -0.00887636]
 [ 0.02473497 -0.00167412]
 [ 0.0154885  -0.00740956]
 [-0.04929382  0.04634152]
 [ 0.01042279 -0.02585928]
 [-0.00034513 -0.01883029]]
21:50  Evaluating kinematic Fisher information on batch 1 / 10
21:50  Evaluating kinematic Fisher information on batch 2 / 10
21:50  Evaluating kinematic Fisher information on batch 3 / 10
21:50  Evaluating kinematic Fisher information on batch 4 / 10
21:50  Evaluating kinematic Fisher information on batch 5 / 10
21:50  Evaluating kinematic Fisher information on batch 6 / 10
21:51  Evaluating kinematic Fisher information on batch 7 / 10
21:51  Evaluating kinematic Fisher inf

In [17]:
means, covs = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_minimal_tight',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_minimal_tight{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_minimal_tight{}.npy'.format(weight_filename), cov)

08:11  Evaluating rate Fisher information
08:11  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_minimal_tight/ensemble.json
08:11  Found ensemble with 10 estimators and expectations [[ 0.07081794 -0.02416615]
 [ 0.09775881  0.07342587]
 [ 0.02155189 -0.1146758 ]
 [-0.25029388  0.11809271]
 [-0.01383648  0.03545747]
 [ 0.03916155 -0.08262159]
 [-0.09130371  0.15367286]
 [-0.05830761 -0.06824306]
 [ 0.01923683  0.00194588]
 [-0.16656899 -0.00395318]]
08:11  Evaluating kinematic Fisher information on batch 1 / 7
08:11  Evaluating kinematic Fisher information on batch 2 / 7
08:12  Evaluating kinematic Fisher information on batch 3 / 7
08:12  Evaluating kinematic Fisher information on batch 4 / 7
08:12  Evaluating kinematic Fisher information on batch 5 / 7
08:12  Evaluating kinematic Fisher information on batch 6 / 7
08:12  Evaluating kinematic Fisher information on batch 7 / 7


## Info in observables (ML)

In [13]:
means, covs = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_resurrection',
    unweighted_x_sample_file = sample_dir + 'test_tight/x_test.npy',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_resurrection_tight{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_resurrection_tight{}.npy'.format(weight_filename), cov)
    

21:51  Evaluating rate Fisher information
21:52  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_resurrection/ensemble.json
21:52  Found ensemble with 10 estimators and expectations [[ 0.06343307 -0.04893372]
 [ 0.09478135 -0.01341373]
 [ 0.04839911 -0.00616729]
 [ 0.01964112  0.05162339]
 [ 0.08938077  0.00986628]
 [ 0.06625167 -0.0093354 ]
 [ 0.03805659 -0.0091226 ]
 [-0.01882457  0.02061494]
 [-0.07291223 -0.07865967]
 [ 0.00173969 -0.04213462]]


## Histogram info

In [14]:
filenames = ['phi_resurrection', 'met', 'ptl', 'pta', 'deltaphi_lv', 'deltaphi_la']
observables = ['phi_resurrection', 'et_miss', 'pt_l1', 'pt_a1', 'deltaphi_l1_met', 'deltaphi_l1_a1']
bins = 25

In [15]:
for filename, obs in zip(filenames, observables):
    print(filename)
    info, cov = fisher_tight.calculate_fisher_information_hist1d(
        theta=[0.,0.],
        luminosity=300000.,
        observable=obs,
        nbins=bins,
        histrange=None
    )
    
    np.save(result_dir + 'information_histo_{}_tight.npy'.format(filename), info)
    np.save(result_dir + 'information_covariance_histo_{}_tight.npy'.format(filename), cov)
    

phi_resurrection
met
ptl
pta
deltaphi_lv
deltaphi_la
